In [1]:
%load_ext tensorboard
%load_ext autoreload

%set_env CUDA_LAUNCH_BLOCKING=0

env: CUDA_LAUNCH_BLOCKING=0


In [2]:
%tensorboard --logdir lightning_logs

In [3]:
%autoreload 2

import pytorch_lightning as lightning

import torch
from torch.utils.data import TensorDataset
from sklearn.datasets import make_moons, make_circles

import numpy as np
import matplotlib.pyplot as plt

from rectflow import RectifyingFlow

In [4]:
plt.rc("figure", dpi=250)
plt.rc("legend", fontsize=6)

In [5]:
accelerator = "gpu"
devices = 1

In [6]:
torch.autograd.set_grad_enabled(False)

In [7]:
n_train = 10000
n_val = 1000

In [8]:
features, labels = make_moons(n_train, shuffle=True, noise=0.05)
noise = np.random.standard_normal((n_train, 2))
features = torch.from_numpy(features.astype(np.float32))
noise = torch.from_numpy(noise.astype(np.float32))

train_data = TensorDataset(features, noise)

features, labels = make_moons(n_val, shuffle=True, noise=0.05)
noise = np.random.standard_normal((n_val, 2))
features = torch.from_numpy(features.astype(np.float32))
noise = torch.from_numpy(noise.astype(np.float32))

val_data = TensorDataset(features, noise)

In [9]:
hparams = dict(
    input_shape=(2,),
    batch_size=4,
    learning_rate=1e-3,
    weight_decay=1e-5,
    integrator="rk45",
    network_widths=[64, 128, 64],
)

In [10]:
model = RectifyingFlow(train_data, val_data, test_data=None, **hparams)

In [11]:
trainer = lightning.Trainer(
    accelerator=accelerator,
    devices=devices,
    benchmark=True,
    max_epochs=100,
)

with torch.autograd.enable_grad():
    model.train()
    trainer.fit(model)

# best_model = model.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
# best_model.eval()

# model = best_model

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | network | Sequential | 16.9 K
---------------------------------------
16.9 K    Trainable params
0         Non-trainable params
16.9 K    Total params
0.068     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

AttributeError: Missing attribute "time_range"

In [ ]:
samples = np.random.standard_normal((1000, 2))
samples = samples.astype(np.float32)
samples = torch.from_numpy(samples)
samples = model.inverse(samples)

plt.scatter(features[:, 0], features[:, 1], s=1, label="val")
plt.scatter(samples[:, 0], samples[:, 1], s=1, label="samples")
plt.legend()

In [ ]:
n, m = 100, 100
x = np.linspace(-3, 3, n)
y = np.linspace(-3, 3, m)

xx, yy = np.meshgrid(x, y)
x, y = xx.flatten(), yy.flatten()
xy = np.stack((x, y), axis=1)
xy = torch.from_numpy(xy).to(torch.float32).to(model.device)

v = model.network(xy)
norm = np.linalg.norm(v, axis=-1)

artist = plt.imshow(norm.reshape(n, m), extent=(x.min(), x.max(), y.min(), y.max()))
plt.colorbar(artist, fraction=0.046, pad=0.04)
plt.quiver(x, y, v[:, 0], v[:, 1], angles="xy", pivot="mid", color="white")